In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):    
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(' ')), 3) * 100


data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))
data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(' '))

##From Starting again in a single code:

def clean_text(text):
    
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106
0,24,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,39,15.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,116,6.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [9]:
def train_Rf(n_est, depth):

    rfc = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rfc.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('EST: {} / Depth: {} --- precision: {}, recall: {}, fscore: {}' .format(n_est, depth, round(precision, 3),
                                                     round(recall, 3),
                                                     round((y_pred==y_test).sum() / len(y_pred), 3)))
    
    

In [10]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_Rf(n_est, depth)

EST: 10 / Depth: 10 --- precision: 1.0, recall: 0.299, fscore: 0.903
EST: 10 / Depth: 20 --- precision: 1.0, recall: 0.617, fscore: 0.947
EST: 10 / Depth: 30 --- precision: 1.0, recall: 0.682, fscore: 0.956
EST: 10 / Depth: None --- precision: 1.0, recall: 0.747, fscore: 0.965
EST: 50 / Depth: 10 --- precision: 1.0, recall: 0.234, fscore: 0.894
EST: 50 / Depth: 20 --- precision: 1.0, recall: 0.61, fscore: 0.946
EST: 50 / Depth: 30 --- precision: 1.0, recall: 0.695, fscore: 0.958
EST: 50 / Depth: None --- precision: 1.0, recall: 0.799, fscore: 0.972
EST: 100 / Depth: 10 --- precision: 1.0, recall: 0.253, fscore: 0.897
EST: 100 / Depth: 20 --- precision: 1.0, recall: 0.591, fscore: 0.943
EST: 100 / Depth: 30 --- precision: 1.0, recall: 0.714, fscore: 0.961
EST: 100 / Depth: None --- precision: 1.0, recall: 0.773, fscore: 0.969


### GRIDSEARCH_CV

In [13]:
##From starting again:


import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):    
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(' ')), 3) * 100


data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))
data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(' '))

##From Starting again in a single code:

def clean_text(text):
    
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

#TFIDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis =1)

#CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_counts = count_vect.fit_transform(data['body_text'])
X_counts_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis =1)

X_counts_feat.head()



,body_len,punct%,0,1,2,3,4,5,6,7,...,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106
0,24,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,39,15.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,116,6.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
rf = RandomForestClassifier()
param = {'n_estimators' : [10, 150, 300],
        'max_depth' : [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_model = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_model.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,42.539197,3.806591,0.511562,0.126563,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978475,0.980269,0.973070,0.968582,0.972147,0.974511,0.004283,1
11,77.109464,13.300998,0.547137,0.077370,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975785,0.976682,0.974865,0.968582,0.973944,0.973972,0.002846,2
10,46.006527,0.487472,0.484055,0.018647,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975785,0.975785,0.974865,0.968582,0.973944,0.973793,0.002693,3
8,83.946028,3.470409,0.747233,0.053676,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977578,0.974888,0.975763,0.967684,0.972147,0.973613,0.003443,4
6,6.134230,0.557144,0.248535,0.042508,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.971300,0.974888,0.971275,0.970377,0.976640,0.972895,0.002429,5


In [16]:
%%time

rf = RandomForestClassifier()
param = {'n_estimators' : [10, 150, 300],
        'max_depth' : [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_model = gs.fit(X_counts_feat, data['label'])
pd.DataFrame(gs_model.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

Wall time: 10min 10s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,41.982602,1.386405,0.515933,0.063954,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976682,0.976682,0.972172,0.967684,0.974843,0.973613,0.003393,1
8,78.532314,0.548944,0.683473,0.014539,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.973991,0.976682,0.974865,0.969479,0.972147,0.973434,0.002458,2
11,79.413741,6.194409,0.648138,0.065420,None,300,"{'max_depth': None, 'n_estimators': 300}",0.974888,0.976682,0.974865,0.967684,0.973046,0.973434,0.003096,2
10,43.689933,0.429446,0.482359,0.036953,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975785,0.976682,0.973968,0.967684,0.970350,0.972895,0.003390,4
4,35.487940,1.194831,0.419926,0.015378,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.975785,0.974888,0.972172,0.966786,0.972147,0.972357,0.003139,5
